Do a random forest return predictor for long time equity screens, maybe including rates/ sectors / commodities / geoploitic indicators. 
Find the % of regimes in the last 30 days that have a higher volatility than the current IV 
Add trend in shannon entropy or/and regime percentages to evaluate recessions / hostile environments beginning. could also look at earnings timestamps affecting options prices 

Want to find stocks with a low hv/iv high regime % in highest stdev state, and high # of days with stdev above avg iv on options 

In [10]:
import yfinance as yf 
from yfinance import EquityQuery
import numpy as np
import pandas as pd
import time


In [11]:
from yfinance import EquityQuery, screen
import yfinance as yf
import pandas as pd
import numpy as np
import datetime
import time

# --- Helper Functions ---

def find_closest_value(data_list, target_value):
    if len(data_list) == 0:
        return None
    return min(data_list, key=lambda x: abs(x - target_value))

def filter_special_expiry(expiry_list):
    """Return the two soonest (earliest) expiry dates from the list."""
    sorted_expiries = sorted(expiry_list)
    return sorted_expiries[:2]

def get_prev_day_price_volume(symbol):
    """Fetch the most recent full day's close price and volume for the symbol."""
    try:
        data = yf.Ticker(symbol).history(period="5d")
        if data.shape[0] < 2:
            return None, None
        prev_day = data.iloc[-2]
        price = float(prev_day['Close'])
        volume = int(prev_day['Volume'])
        return price, volume
    except Exception as e:
        print(f"Error fetching previous day price/volume for {symbol}: {e}")
        return None, None

def get_historical_volatility(symbol, window=20):
    try:
        data = yf.Ticker(symbol).history(period=f"{window*2}d")
        if data.shape[0] < window + 1:
            return None
        data = data['Close'].dropna()
        log_returns = np.log(data / data.shift(1)).dropna()
        vol = log_returns[-window:].std() * np.sqrt(252)
        return vol
    except Exception as e:
        print(f"Error calculating historical volatility for {symbol}: {e}")
        return None

def get_options(symbol, max_retries=3, retry_delay=60):
    for attempt in range(max_retries):
        try:
            stock = yf.Ticker(symbol)
            options_chain = stock.options
            if not options_chain:
                return pd.DataFrame(columns=['strike', 'impliedVolatility', 'openInterest', 'symbol'])
            filtered_expiries = filter_special_expiry(options_chain)
            options_data = []
            current_price, _ = get_prev_day_price_volume(symbol)
            if current_price is None:
                continue
            for expiry in filtered_expiries:
                chain = stock.option_chain(expiry)
                for opt_type, df in [('call', chain.calls), ('put', chain.puts)]:
                    if not df.empty:
                        closest_strike = find_closest_value(df['strike'], current_price)
                        filtered = df[df['strike'] == closest_strike].copy()
                        filtered['type'] = opt_type
                        filtered['expiry'] = expiry
                        filtered['symbol'] = symbol
                        options_data.append(filtered)
            if options_data:
                result = pd.concat(options_data, ignore_index=True)
                result = result[result['openInterest'] > 500]
                return result[['strike', 'impliedVolatility', 'openInterest', 'symbol']]
            else:
                return pd.DataFrame(columns=['strike', 'impliedVolatility', 'openInterest', 'symbol'])
        except Exception as e:
            print(f"Error fetching options for {symbol}: {e}")
            if attempt < max_retries - 1:
                time.sleep(retry_delay)
                continue
            else:
                print(f"Max retries reached for {symbol}.")
                return pd.DataFrame(columns=['strike', 'impliedVolatility', 'openInterest', 'symbol'])
    return pd.DataFrame(columns=['strike', 'impliedVolatility', 'openInterest', 'symbol'])

# --- Screen Query ---
screen_query = EquityQuery("and", [
    EquityQuery("is-in", ["region", "us"]),
    EquityQuery("is-in", ["sector",
        "Technology",
        "Financial Services",
        "Consumer Cyclical",
        "Communication Services",
        "Basic Materials",
        "Industrials"
    ]),
    EquityQuery("GTE", ["eodprice", 0.50]),
    EquityQuery("GTE", ["avgdailyvol3m", 100000]),
    EquityQuery("LTE", ["short_percentage_of_float.value", 100])
])

# --- Batch Processing Function ---
def process_batch(offset, filterdf, options):
    batch_size = 100
    screener = screen(
        screen_query,
        sortField="avgdailyvol3m",
        sortAsc=False,
        size=batch_size,
        offset=offset
    )
    stocks = screener.get('quotes', [])
    if not stocks:
        print("No more stocks to load.")
        return filterdf, options, offset

    current_filterdf = pd.DataFrame(stocks)

    # --- Add price*volume filter for most recent full day ---
    current_filterdf['prev_day_price'] = None
    current_filterdf['prev_day_volume'] = None
    current_filterdf['price_volume'] = None

    for idx, row in current_filterdf.iterrows():
        symbol = row['symbol']
        price, volume = get_prev_day_price_volume(symbol)
        current_filterdf.at[idx, 'prev_day_price'] = price
        current_filterdf.at[idx, 'prev_day_volume'] = volume
        if price is not None and volume is not None:
            current_filterdf.at[idx, 'price_volume'] = price * volume

    current_filterdf = current_filterdf[current_filterdf['price_volume'] > 100_000_000]

    if current_filterdf.empty:
        print("No stocks passed the price*volume filter in this batch.")
        offset += batch_size
        return filterdf, options, offset

    current_filterdf['historical_volatility'] = None
    for idx, row in current_filterdf.iterrows():
        symbol = row['symbol']
        hv = get_historical_volatility(symbol, window=20)
        current_filterdf.at[idx, 'historical_volatility'] = hv

    filterstocks = current_filterdf['symbol'].tolist()
    options_list = []
    for symbol in filterstocks:
        options_data = get_options(symbol)
        options_list.append(options_data)

    if options_list:
        current_options = pd.concat(options_list, ignore_index=True)
        options = pd.concat([options, current_options], ignore_index=True)
    else:
        current_options = pd.DataFrame(columns=['strike', 'impliedVolatility', 'openInterest', 'symbol'])

    if not current_options.empty:
        qualifying_symbols = set(current_options['symbol'].str.upper().unique())
        current_filterdf = current_filterdf[current_filterdf['symbol'].str.upper().isin(qualifying_symbols)]
    else:
        current_filterdf = current_filterdf.iloc[0:0]

    if current_filterdf.empty:
        print("No stocks passed the open interest filter in this batch.")
        offset += batch_size
        return filterdf, options, offset

    if not options.empty and 'symbol' in options.columns and 'impliedVolatility' in options.columns:
        avg_iv = options.groupby('symbol')['impliedVolatility'].mean().reset_index()
        avg_iv.columns = ['symbol', 'avg_iv']
    else:
        avg_iv = pd.DataFrame(columns=['symbol', 'avg_iv'])

    current_filterdf['symbol'] = current_filterdf['symbol'].astype(str).str.upper()
    avg_iv['symbol'] = avg_iv['symbol'].astype(str).str.upper()
    current_filterdf = current_filterdf.merge(avg_iv, on='symbol', how='left')

    if 'avg_iv' in current_filterdf.columns and 'historical_volatility' in current_filterdf.columns:
        current_filterdf["iv/hv_ratio"] = current_filterdf["avg_iv"] / current_filterdf["historical_volatility"]
    else:
        current_filterdf["iv/hv_ratio"] = None

    filterdf = pd.concat([filterdf, current_filterdf], ignore_index=True)

    offset += batch_size
    return filterdf, options, offset

# --- Main loop ---
offset = 0
filterdf = pd.DataFrame()
options = pd.DataFrame()

filterdf, options, offset = process_batch(offset, filterdf, options)
print("Initial batch:")
print(filterdf)

while True:
    user_input = input("Process next 100 stocks? (y/n): ")
    if user_input.lower() != 'y':
        break
    filterdf, options, offset = process_batch(offset, filterdf, options)
    print("Updated results:")
    print(filterdf)


Initial batch:
   language region quoteType typeDisp         quoteSourceName  triggerable  \
0     en-US     US    EQUITY   Equity  Nasdaq Real Time Price         True   
1     en-US     US    EQUITY   Equity  Nasdaq Real Time Price         True   
2     en-US     US    EQUITY   Equity  Nasdaq Real Time Price        False   
3     en-US     US    EQUITY   Equity  Nasdaq Real Time Price         True   
4     en-US     US    EQUITY   Equity  Nasdaq Real Time Price        False   
5     en-US     US    EQUITY   Equity  Nasdaq Real Time Price         True   
6     en-US     US    EQUITY   Equity  Nasdaq Real Time Price         True   
7     en-US     US    EQUITY   Equity  Nasdaq Real Time Price         True   
8     en-US     US    EQUITY   Equity  Nasdaq Real Time Price         True   
9     en-US     US    EQUITY   Equity  Nasdaq Real Time Price         True   
10    en-US     US    EQUITY   Equity  Nasdaq Real Time Price         True   
11    en-US     US    EQUITY   Equity  Nasdaq Rea

In [12]:
import yfinance as yf
import numpy as np
from hmmlearn.hmm import GaussianHMM
from tqdm import tqdm

def get_last_120_returns(symbol):
    """Get last 120 days of daily log returns for a symbol."""
    try:
        data = yf.Ticker(symbol).history(period="130d")
        closes = data['Close'].dropna()
        if len(closes) < 121:
            return None
        returns = np.log(closes / closes.shift(1)).dropna()
        return returns[-120:]
    except Exception as e:
        print(f"Error fetching data for {symbol}: {e}")
        return None

def fit_hmm_and_stats(returns, n_states=3):
    """Fit HMM to returns, return means, stds, and state sequence."""
    model = GaussianHMM(
        n_components=n_states,
        covariance_type="diag",
        n_iter=500,
        tol=1e-2,
        random_state=42,
        verbose=False
    )
    X = returns.values.reshape(-1, 1)
    model.fit(X)
    hidden_states = model.predict(X)
    means = model.means_.flatten()
    stds = np.sqrt(np.array([np.diag(cov)[0] for cov in model.covars_]))
    return hidden_states, means, stds

def hmm_analysis(symbol):
    returns = get_last_120_returns(symbol)
    if returns is None:
        return {
            "hmm_mean_0": None,
            "hmm_std_0": None,
            "hmm_mean_1": None,
            "hmm_std_1": None,
            "hmm_mean_2": None,
            "hmm_std_2": None,
            "pct_last30_in_highest_stdev_state": None,
            "highest_stdev_state": None
        }
    hidden_states, means, stds = fit_hmm_and_stats(returns, n_states=3)
    # Identify the state with the highest stdev
    high_stdev_state = int(np.argmax(stds))
    last30 = hidden_states[-30:]
    pct_high = np.mean(last30 == high_stdev_state) * 100
    return {
        "hmm_mean_0": means[0],
        "hmm_std_0": stds[0],
        "hmm_mean_1": means[1],
        "hmm_std_1": stds[1],
        "hmm_mean_2": means[2],
        "hmm_std_2": stds[2],
        "pct_last30_in_highest_stdev_state": pct_high,
        "highest_stdev_state": high_stdev_state
    }

# --- Apply to filterdf ---

filterdf['symbol'] = filterdf['symbol'].astype(str).str.upper()

for col in [
    'hmm_mean_0','hmm_std_0',
    'hmm_mean_1','hmm_std_1',
    'hmm_mean_2','hmm_std_2',
    'pct_last30_in_highest_stdev_state',
    'highest_stdev_state'
]:
    filterdf[col] = None

for idx, row in tqdm(filterdf.iterrows(), total=len(filterdf)):
    symbol = row['symbol']
    hmm_stats = hmm_analysis(symbol)
    for col in hmm_stats:
        filterdf.at[idx, col] = hmm_stats[col]

# filterdf now has the HMM columns for states 0, 1, 2, and the % of last 30 closes in the highest stdev state.


  0%|          | 0/79 [00:00<?, ?it/s]

Model is not converging.  Current: 250.28804324156005 is not greater than 250.29019346407003. Delta is -0.0021502225099823136
100%|██████████| 79/79 [00:18<00:00,  4.18it/s]


In [13]:
N = 252  # Or use 261, depending on your convention

def pct_regimes_higher_than_iv(hidden_states, stds, avg_iv):
    """% of regimes in last 30 days with annualized stdev higher than avg_iv."""
    last30 = hidden_states[-30:]
    # Annualize regime stdevs
    stds_annualized = stds * np.sqrt(N)
    last30_vols_annualized = stds_annualized[last30]
    pct_higher = np.mean(last30_vols_annualized > avg_iv) * 100
    return pct_higher

filterdf['pct_regimes_last30_higher_than_avg_iv'] = None

for idx, row in tqdm(filterdf.iterrows(), total=len(filterdf)):
    symbol = row['symbol']
    avg_iv = row['avg_iv']
    returns = get_last_120_returns(symbol)
    if returns is None or avg_iv is None:
        filterdf.at[idx, 'pct_regimes_last30_higher_than_avg_iv'] = None
        continue
    hidden_states, means, stds = fit_hmm_and_stats(returns, n_states=3)
    pct_higher = pct_regimes_higher_than_iv(hidden_states, stds, avg_iv)
    filterdf.at[idx, 'pct_regimes_last30_higher_than_avg_iv'] = pct_higher


  0%|          | 0/79 [00:00<?, ?it/s]

Model is not converging.  Current: 250.28804324156005 is not greater than 250.29019346407003. Delta is -0.0021502225099823136
100%|██████████| 79/79 [00:19<00:00,  4.04it/s]


In [14]:
import numpy as np
import pandas as pd
import yfinance as yf
from scipy.stats import entropy
from hmmlearn.hmm import GaussianHMM
from statsmodels.tsa.stattools import adfuller

def shannon_entropy(series, bins=10):
    hist, _ = np.histogram(series, bins=bins, density=True)
    hist = hist[hist > 0]
    return entropy(hist, base=2)

def hmm_regime_switch_prob_and_count(returns, n_states=2):
    returns = returns.values.reshape(-1, 1)
    model = GaussianHMM(n_components=n_states, covariance_type="diag", n_iter=100)
    model.fit(returns)
    hidden_states = model.predict(returns)
    last_state = hidden_states[-1]
    transmat = model.transmat_
    switch_prob = 1 - transmat[last_state, last_state]
    regime_switch_count = np.sum(hidden_states[1:] != hidden_states[:-1])
    return switch_prob, regime_switch_count

entropy_list = []
regimeswitchprob_list = []
regimeswitchcount_list = []
adf_pvalue_list = []
avg_return_list = []

for ticker in filterdf['symbol']:
    try:
        # Download last 7 days of 1-minute data
        df = yf.download(ticker, period="7d", interval="1m", progress=False)
        closes = df['Close'].dropna()
        # Make sure you have enough data (e.g., at least 1000 minutes)
        if len(closes) < 1000:
            entropy_list.append(np.nan)
            regimeswitchprob_list.append(np.nan)
            regimeswitchcount_list.append(np.nan)
            adf_pvalue_list.append(np.nan)
            avg_return_list.append(np.nan)
            continue
        log_returns = np.log(closes).diff().dropna()         # For entropy, HMM, etc.
        simple_returns = closes.pct_change().dropna()        # For avg return only

        # Shannon entropy (log returns)
        ent = shannon_entropy(log_returns)
        entropy_list.append(ent)
        # HMM regime switch probability and count (log returns)
        switch_prob, switch_count = hmm_regime_switch_prob_and_count(log_returns)
        regimeswitchprob_list.append(round(switch_prob * 100, 2))  # as percentage
        regimeswitchcount_list.append(int(switch_count))
        # ADF test p-value on last 7 days of prices (not returns)
        adf_pvalue = adfuller(closes)[1]
        adf_pvalue_list.append(adf_pvalue)
        # Average return over last 7 days (mean of simple returns)
        avg_return = simple_returns.mean()
        avg_return_list.append(avg_return)
    except Exception as e:
        entropy_list.append(np.nan)
        regimeswitchprob_list.append(np.nan)
        regimeswitchcount_list.append(np.nan)
        adf_pvalue_list.append(np.nan)
        avg_return_list.append(np.nan)


filterdf['ShannonEntropy'] = entropy_list
filterdf['Regimeswitchprob'] = regimeswitchprob_list
filterdf['Regimeswitchcount'] = regimeswitchcount_list
filterdf['ADF_Pvalue'] = adf_pvalue_list
filterdf['AvgReturn_7d'] = avg_return_list


Model is not converging.  Current: 11317.616355197464 is not greater than 11318.87451379854. Delta is -1.2581586010765022
Model is not converging.  Current: 13773.933746580622 is not greater than 13775.376309052843. Delta is -1.442562472220743
Model is not converging.  Current: 14028.75770316379 is not greater than 14030.157983025352. Delta is -1.4002798615620122
Model is not converging.  Current: 13951.307513971167 is not greater than 13951.322339935436. Delta is -0.014825964268311509
Model is not converging.  Current: 13914.709259650219 is not greater than 13916.595369124567. Delta is -1.8861094743479043
Model is not converging.  Current: 13523.684640400046 is not greater than 13526.443661365221. Delta is -2.7590209651752957
Model is not converging.  Current: 13564.66683990842 is not greater than 13568.522645832987. Delta is -3.8558059245679033
Model is not converging.  Current: 13862.650379939389 is not greater than 13867.532912505118. Delta is -4.8825325657289795
Model is not conve

In [15]:
# Show all rows and columns
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
filterdf["Shannon/IV"] = filterdf["ShannonEntropy"] / filterdf["avg_iv"]
display(filterdf)


,language,region,quoteType,typeDisp,quoteSourceName,triggerable,customPriceAlertConfidence,currency,bid,ask,regularMarketChangePercent,exchange,fiftyTwoWeekHigh,fiftyTwoWeekLow,averageAnalystRating,dividendYield,shortName,corporateActions,fiftyTwoWeekLowChange,fiftyTwoWeekLowChangePercent,fiftyTwoWeekRange,fiftyTwoWeekHighChange,fiftyTwoWeekHighChangePercent,fiftyTwoWeekChangePercent,dividendDate,earningsTimestamp,earningsTimestampStart,earningsTimestampEnd,earningsCallTimestampStart,earningsCallTimestampEnd,isEarningsDateEstimate,trailingAnnualDividendRate,trailingPE,dividendRate,trailingAnnualDividendYield,marketState,epsTrailingTwelveMonths,epsForward,epsCurrentYear,priceEpsCurrentYear,sharesOutstanding,bookValue,fiftyDayAverage,fiftyDayAverageChange,fiftyDayAverageChangePercent,twoHundredDayAverage,twoHundredDayAverageChange,twoHundredDayAverageChangePercent,marketCap,forwardPE,priceToBook,sourceInterval,exchangeDataDelayedBy,exchangeTimezoneName,exchangeTimezoneShortName,gmtOffSetMilliseconds,prevName,nameChangeDate,esgPopulated,tradeable,cryptoTradeable,hasPrePostMarketData,firstTradeDateMilliseconds,priceHint,postMarketChangePercent,postMarketTime,postMarketPrice,postMarketChange,regularMarketChange,regularMarketTime,regularMarketPrice,regularMarketDayHigh,regularMarketDayRange,regularMarketDayLow,regularMarketVolume,regularMarketPreviousClose,bidSize,askSize,market,messageBoardId,fullExchangeName,longName,financialCurrency,regularMarketOpen,averageDailyVolume3Month,averageDailyVolume10Day,displayName,symbol,ipoExpectedDate,prev_day_price,prev_day_volume,price_volume,historical_volatility,avg_iv,iv/hv_ratio,hmm_mean_0,hmm_std_0,hmm_mean_1,hmm_std_1,hmm_mean_2,hmm_std_2,pct_last30_in_highest_stdev_state,highest_stdev_state,pct_regimes_last30_higher_than_avg_iv,ShannonEntropy,Regimeswitchprob,Regimeswitchcount,ADF_Pvalue,AvgReturn_7d,Shannon/IV
0,en-US,US,EQUITY,Equity,Nasdaq Real Time Price,True,HIGH,USD,172.00,175.00,1.733230,NMS,174.250,86.620000,1.4 - Strong Buy,0.02,NVIDIA Corporation,[],87.120000,1.005772,86.62 - 174.25,-0.509995,-0.002927,53.670620,1.751501e+09,1.756325e+09,1756324800,1756324800,1.748466e+09,1.748466e+09,False,0.040,55.864956,0.04,0.000234,POSTPOST,3.11,4.12,4.31755,40.240414,24387600384,3.438,149.4882,24.251800,0.162232,133.420650,40.319350,0.302197,4237101891584,42.169907,50.535200,15,0,America/New_York,EDT,-14400000,Usual Stablecoin,2025-07-24,False,False,False,True,917015400000,2,-0.089213,1753401598,173.5850,-0.154999,2.960010,1753387200,173.74,173.8300,171.3 - 173.83,171.3000,127727175,170.780,3,1,us_market,finmb_32307,NasdaqGS,NVIDIA Corporation,USD,172.450,196315659,162624090,NVIDIA,NVDA,NaN,170.779999,154082200,26314157927.911377,0.260627,0.357428,1.371416,0.000174,0.033218,0.008021,0.024142,-0.011813,0.083891,0.0,2,100.0,0.506789,0.06,4,0.446920,Ticker NVDA 0.000005 dtype: float64,1.417876
1,en-US,US,EQUITY,Equity,Nasdaq Real Time Price,True,HIGH,USD,2.95,2.99,-2.922080,NMS,4.430,1.930000,3.1 - Hold,NaN,"Lucid Group, Inc.",[],1.060000,0.549223,1.93 - 4.43,-1.440000,-0.325056,-11.538464,NaN,1.754424e+09,1754424000,1754424000,1.754429e+09,1.754429e+09,False,0.000,NaN,NaN,0.000000,POSTPOST,-1.19,-0.88,-0.89314,-3.347739,3050259968,1.044,2.4094,0.580600,0.240973,2.540325,0.449675,0.177015,9120277504,-3.397727,2.863985,15,0,America/New_York,EDT,-14400000,NaN,NaN,False,False,False,True,1600435800000,4,-0.334448,1753401588,2.9800,-0.010000,-0.090000,1753387202,2.99,3.0700,2.91 - 3.07,2.9100,92868184,3.080,67,120,us_market,finmb_83747444,NasdaqGS,"Lucid Group, Inc.",USD,3.030,153236545,234252100,Lucid,LCID,2021-07-26,3.08,181511400,559055098.151779,1.286375,1.029299,0.800155,-0.008318,0.03996,0.004575,0.051109,0.302688,0.106872,3.333333,2,3.333333,0.135699,0.58,20,0.155076,Ticker LCID 0.000113 dtype: float64,0.131837
2,en-US,US,EQUITY,Equity,Nasdaq Real Time Price,False,LOW,USD,1.74,1.90,-2.209940,NCM,3.320,0.690000,2.9 - Hold,NaN,"Plug Power, Inc.",[],1.080000,1.565217,0.69 - 3.32,-1.550000,-0.4

In [ ]:
import nest_asyncio
nest_asyncio.apply()
import asyncio
from ib_insync import *
import pandas as pd
import numpy as np
import ta
import plotly.graph_objects as go

"""
Filter parameters
-----------------
VOL_WINDOW:       ATR period (bars) for volatility calculation (consider higher for daily)
VOL_THRESHOLD:    Minimum ATR percent (ATR/close) for a bar to be considered volatile enough for trading (e.g., 0.01 = 1%)
REGIME_THRESHOLD: Rolling mean of returns must be above this (e.g., -0.1) to be considered "trending" regime
ADX_PERIOD:       ADX calculation period (bars)
ADX_THRESHOLD:    Minimum ADX value to confirm market is trending (20 = classic)
SMA_PERIOD:       SMA bars for trend filter (only long if above, short if below)
"""

# ========= CHART SETTINGS =========
BAR_SIZE = '1 day'      # IBKR bar size for daily candles
DURATION = '5 Y'        # How far back to get data (e.g., '5 Y' = 5 years)
NUM_BARS = 2000         # Approximate daily bars in 5 years
# ================================

# Filter tuning (adjust to your needs)
VOL_WINDOW       = 30        # ATR lookback period (bars); for daily, 14-30 is common
VOL_THRESHOLD    = 0.001     # Minimum ATR% for volatility filter (e.g., 0.01=1%)
REGIME_THRESHOLD = -0.1      # Regime filter threshold
ADX_PERIOD       = 14        # ADX lookback window (bars)
ADX_THRESHOLD    = 20        # Minimum ADX to allow signals (trend filter)
SMA_PERIOD       = 200       # SMA filter period (bars)

EXCHANGE    = 'SMART'
CURRENCY    = 'USD'
LOOKAHEAD   = 4
K_NEIGHBORS = 8

FEATURES = ['RSI', 'ADX', 'CCI', 'MACD']


# ==== Data Fetch Function ====
async def fetch_ibkr_stock(symbol, exchange, currency, bar_size, duration, num_bars):
    """Fetches OHLCV bars from IBKR using specified bar size/duration."""
    ib = IB()
    await ib.connectAsync('127.0.0.1', 7497, clientId=np.random.randint(1000, 10000))
    contract = Stock(symbol, exchange, currency)
    await ib.qualifyContractsAsync(contract)
    bars = await ib.reqHistoricalDataAsync(
        contract=contract,
        endDateTime='',
        durationStr=duration,
        barSizeSetting=bar_size,
        whatToShow='TRADES',
        useRTH=True,
        formatDate=1,
        keepUpToDate=False
    )
    df = util.df(bars)
    df.set_index('date', inplace=True)
    df = df.tail(num_bars)
    ib.disconnect()
    return df


# ==== ML Functions and Filters ====
def lorentzian_distance(a, b):
    return np.sum(np.log(1 + np.abs(a - b)))


def volatility_filter(df, vol_window, vol_thresh):
    atr = ta.volatility.AverageTrueRange(df['high'], df['low'], df['close'], window=vol_window).average_true_range()
    atr_pct = atr / df['close']
    return atr_pct > vol_thresh


def regime_filter(df, threshold):
    close_returns = df['close'].pct_change()
    regime = close_returns.rolling(window=20).mean()
    return regime > threshold


def adx_filter(df, adx_period, adx_threshold):
    adx = ta.trend.ADXIndicator(df['high'], df['low'], df['close'], window=adx_period).adx()
    return adx > adx_threshold


def classify_lorentzian_knn_with_filters(
        df, features, sma_period,
        vol_window, vol_thresh,
        regime_thresh, adx_period, adx_thresh,
        n_neighbors=8, lookahead=4, max_bars_back=500):
    pred = np.full(len(df), np.nan)
    neighbors_info = {}

    closes = df['close'].values
    feature_mat = df[features].values
    sma = df['SMA'].values

    vol_filter = volatility_filter(df, vol_window, vol_thresh)
    regime_filt = regime_filter(df, regime_thresh)
    adx_filt = adx_filter(df, adx_period, adx_thresh)

    length = len(df)

    # Main loop - exclude last 'lookahead' bars
    for idx in range(max_bars_back, length - lookahead):
        if np.isnan(sma[idx]) or closes[idx] == 0:
            continue
        if not (vol_filter.iloc[idx] and regime_filt.iloc[idx] and adx_filt.iloc[idx]):
            continue
        anchor_start = max(0, idx - max_bars_back)
        anchor_indices = np.arange(anchor_start, idx)
        anchor_feats = feature_mat[anchor_indices, :]
        target = feature_mat[idx]
        dists = np.array([lorentzian_distance(target, anchor_feats[j]) for j in range(anchor_feats.shape[0])])
        if len(dists) < n_neighbors:
            continue
        knn_indices = dists.argsort()[:n_neighbors]
        valid = anchor_indices[knn_indices] + lookahead < length
        selected = anchor_indices[knn_indices][valid]
        if len(selected) == 0:
            continue
        y_train = (closes[selected + lookahead] > closes[selected]).astype(int) - \
                  (closes[selected + lookahead] < closes[selected]).astype(int)
        vote = y_train.sum()
        raw_signal = np.sign(vote) if vote != 0 else 0
        if raw_signal == 1 and closes[idx] > sma[idx]:
            pred[idx] = 1
        elif raw_signal == -1 and closes[idx] < sma[idx]:
            pred[idx] = -1
        else:
            pred[idx] = 0
        neighbors_info[idx] = y_train.tolist()

    # Explicit classification for last 'lookahead' bars
    for idx in range(length - lookahead, length):
        if np.isnan(sma[idx]) or closes[idx] == 0:
            continue
        if not (vol_filter.iloc[idx] and regime_filt.iloc[idx] and adx_filt.iloc[idx]):
            continue
        anchor_start = max(0, idx - max_bars_back)
        anchor_indices = np.arange(anchor_start, idx)
        anchor_feats = feature_mat[anchor_indices, :]
        target = feature_mat[idx]
        dists = np.array([lorentzian_distance(target, anchor_feats[j]) for j in range(anchor_feats.shape[0])])
        if len(dists) < n_neighbors:
            continue
        knn_indices = dists.argsort()[:n_neighbors]
        valid = anchor_indices[knn_indices] + lookahead < length
        selected = anchor_indices[knn_indices][valid]
        if len(selected) == 0:
            continue
        y_train = (closes[selected + lookahead] > closes[selected]).astype(int) - \
                  (closes[selected + lookahead] < closes[selected]).astype(int)
        vote = y_train.sum()
        raw_signal = np.sign(vote) if vote != 0 else 0
        if raw_signal == 1 and closes[idx] > sma[idx]:
            pred[idx] = 1
        elif raw_signal == -1 and closes[idx] < sma[idx]:
            pred[idx] = -1
        else:
            pred[idx] = 0
        neighbors_info[idx] = y_train.tolist()

    return pred, neighbors_info


# ==== Process symbols from filterdf ====
async def process_symbols_with_knn(filterdf):
    filterdf['pct_buy_last_8d'] = np.nan
    filterdf['pct_sell_last_8d'] = np.nan

    for i, row in filterdf.iterrows():
        symbol = row['symbol']  # Adjust if your symbol column is named differently
        print(f"Processing {symbol} ...")

        df = await fetch_ibkr_stock(symbol, EXCHANGE, CURRENCY, BAR_SIZE, DURATION, NUM_BARS)

        # Feature Engineering
        df['RSI'] = ta.momentum.RSIIndicator(df['close'], window=14).rsi()
        df['ADX'] = ta.trend.ADXIndicator(df['high'], df['low'], df['close'], window=14).adx()
        df['CCI'] = ta.trend.CCIIndicator(df['high'], df['low'], df['close'], window=20).cci()
        df['SMA'] = ta.trend.SMAIndicator(df['close'], window=SMA_PERIOD).sma_indicator()
        macd_ind = ta.trend.MACD(df['close'], window_slow=26, window_fast=12, window_sign=9)
        df['MACD'] = macd_ind.macd()
        df['MACD_signal'] = macd_ind.macd_signal()
        df['MACD_hist'] = macd_ind.macd_diff()
        df = df.dropna().reset_index()

        preds, _ = classify_lorentzian_knn_with_filters(
            df, FEATURES, SMA_PERIOD, VOL_WINDOW, VOL_THRESHOLD,
            REGIME_THRESHOLD, ADX_PERIOD, ADX_THRESHOLD,
            n_neighbors=K_NEIGHBORS, lookahead=LOOKAHEAD, max_bars_back=500
        )
        df['raw_pred'] = preds

        # Calculate % buy and sell signals in the last 8 days of predictions
        last_8 = df['raw_pred'].tail(8)
        pct_buy = (last_8 == 1).sum() / len(last_8) * 100 if len(last_8) > 0 else 0
        pct_sell = (last_8 == -1).sum() / len(last_8) * 100 if len(last_8) > 0 else 0

        filterdf.at[i, 'pct_buy_last_8d'] = pct_buy
        filterdf.at[i, 'pct_sell_last_8d'] = pct_sell

    return filterdf


# --- Example usage ---
# Make sure you have your filter dataframe loaded with a 'symbol' column
# Example:
# filterdf = pd.DataFrame({'symbol': ['GOOG', 'AAPL', 'MSFT']})

if __name__ == '__main__':
    import sys

    # Replace or load your filter dataframe here
    # For demo, you can define it manually or read from CSV

    df_results = asyncio.run(process_symbols_with_knn(filterdf))
    print(df_results[['symbol', 'pct_buy_last_8d', 'pct_sell_last_8d']])


Processing NVDA ...
Processing LCID ...
Processing PLUG ...
Processing TSLA ...
Processing F ...
Processing PLTR ...
Processing INTC ...
Processing SOFI ...
Processing AAL ...
Processing AAPL ...
Processing APLD ...
Processing MARA ...
Processing AMD ...
Processing SMCI ...
Processing HOOD ...
Processing NIO ...
Processing AMZN ...
Processing GOOGL ...
Processing BAC ...
Processing RIOT ...
Processing SNAP ...
Processing VALE ...
Processing QS ...
Processing CLF ...
Processing BTBT ...
Processing AMCR ...
Processing T ...
Processing ITUB ...
Processing QUBT ...
Processing HBAN ...
Processing CLSK ...
Processing CCL ...
Processing RUN ...
Processing HL ...
Processing MU ...
Processing SBET ...
Processing UBER ...
Processing AVGO ...
Processing MRVL ...
Processing MSFT ...
Processing CSCO ...
Processing CMCSA ...
Processing STLA ...
Processing NOK ...
Processing HPE ...
Processing WFC ...
Processing VZ ...
Processing ADT ...
Processing NKE ...
Processing LYFT ...
Processing KEY ...
Proce

In [ ]:
filterdf

,language,region,quoteType,typeDisp,quoteSourceName,triggerable,customPriceAlertConfidence,hasPrePostMarketData,firstTradeDateMilliseconds,priceHint,postMarketChangePercent,postMarketTime,postMarketPrice,postMarketChange,regularMarketChange,regularMarketTime,regularMarketPrice,regularMarketDayHigh,regularMarketDayRange,regularMarketDayLow,regularMarketVolume,regularMarketPreviousClose,bidSize,askSize,market,messageBoardId,fullExchangeName,longName,financialCurrency,regularMarketOpen,averageDailyVolume3Month,averageDailyVolume10Day,corporateActions,fiftyTwoWeekLowChange,fiftyTwoWeekLowChangePercent,fiftyTwoWeekRange,fiftyTwoWeekHighChange,fiftyTwoWeekHighChangePercent,fiftyTwoWeekChangePercent,dividendDate,earningsTimestamp,earningsTimestampStart,earningsTimestampEnd,earningsCallTimestampStart,earningsCallTimestampEnd,isEarningsDateEstimate,trailingAnnualDividendRate,trailingPE,dividendRate,trailingAnnualDividendYield,marketState,epsTrailingTwelveMonths,epsForward,epsCurrentYear,priceEpsCurrentYear,sharesOutstanding,bookValue,fiftyDayAverage,fiftyDayAverageChange,fiftyDayAverageChangePercent,twoHundredDayAverage,twoHundredDayAverageChange,twoHundredDayAverageChangePercent,marketCap,forwardPE,priceToBook,sourceInterval,exchangeDataDelayedBy,exchangeTimezoneName,exchangeTimezoneShortName,gmtOffSetMilliseconds,prevName,nameChangeDate,esgPopulated,tradeable,cryptoTradeable,currency,bid,ask,exchange,fiftyTwoWeekHigh,fiftyTwoWeekLow,averageAnalystRating,dividendYield,shortName,regularMarketChangePercent,displayName,symbol,ipoExpectedDate,prev_day_price,prev_day_volume,price_volume,historical_volatility,avg_iv,iv/hv_ratio,hmm_mean_0,hmm_std_0,hmm_mean_1,hmm_std_1,hmm_mean_2,hmm_std_2,pct_last30_in_highest_stdev_state,highest_stdev_state,pct_regimes_last30_higher_than_avg_iv,ShannonEntropy,Regimeswitchprob,Regimeswitchcount,ADF_Pvalue,AvgReturn_7d,Shannon/IV,pct_buy_last_8d,pct_sell_last_8d
0,en-US,US,EQUITY,Equity,Nasdaq Real Time Price,True,HIGH,True,917015400000,2,-0.089213,1753401598,173.5850,-0.154999,2.960010,1753387200,173.74,173.8300,171.3 - 173.83,171.300,127727175,170.780,3,1,us_market,finmb_32307,NasdaqGS,NVIDIA Corporation,USD,172.450,196315659,162624090,[],87.120000,1.005772,86.62 - 174.25,-0.509995,-0.002927,53.670620,1.751501e+09,1756324800,1756324800,1756324800,1748466000,1748466000,False,0.040,55.864956,0.04,0.000234,POSTPOST,3.11,4.12,4.31755,40.240414,24387600384,3.438,149.4882,24.251800,0.162232,133.420650,40.319350,0.302197,4237101891584,42.169907,50.535200,15,0,America/New_York,EDT,-14400000,Usual Stablecoin,2025-07-24,False,False,False,USD,172.00,175.00,NMS,174.25,86.62,1.4 - Strong Buy,0.02,NVIDIA Corporation,1.733230,NVIDIA,NVDA,NaN,170.779999,154082200,26314157927.911377,0.260627,0.357428,1.371416,0.000174,0.033218,0.008021,0.024142,-0.011813,0.083891,0.0,2,100.0,0.506789,0.07,4,0.446920,Ticker NVDA 0.000005 dtype: float64,1.417876,62.5,0.0
1,en-US,US,EQUITY,Equity,Nasdaq Real Time Price,True,HIGH,True,1600435800000,4,-0.334448,1753401588,2.9800,-0.010000,-0.090000,1753387202,2.99,3.0700,2.91 - 3.07,2.910,92868184,3.080,67,120,us_market,finmb_83747444,NasdaqGS,"Lucid Group, Inc.",USD,3.030,153236545,234252100,[],1.060000,0.549223,1.93 - 4.43,-1.440000,-0.325056,-11.538464,NaN,1754424000,1754424000,1754424000,1754429400,1754429400,False,0.000,NaN,NaN,0.000000,POSTPOST,-1.19,-0.88,-0.89314,-3.347739,3050259968,1.044,2.4094,0.580600,0.240973,2.540325,0.449675,0.177015,9120277504,-3.397727,2.863985,15,0,America/New_York,EDT,-14400000,NaN,NaN,False,False,False,USD,2.95,2.99,NMS,4.43,1.93,3.1 - Hold,NaN,"Lucid Group, Inc.",-2.922080,Lucid,LCID,2021-07-26,3.08,181511400,559055098.151779,1.286375,1.029299,0.800155,-0.008318,0.03996,0.004575,0.051109,0.302688,0.106872,3.333333,2,3.333333,0.135699,0.58,20,0.155076,Ticker LCID 0.000113 dtype: float64,0.131837,50.0,0.0
2,en-US,US,EQUITY,Equity,Nasdaq Real Time Price,False,LOW,True,941203800000,4,-0.016945,1753401569,1.7697,-0.000300,-0.040000,1753387200,1.77,1.9900,1.74 -

In [ ]:
import datetime

today = datetime.datetime.today().strftime('%Y-%m-%d')
filename = f'/Users/nshaffer/Desktop/equity_vol_screen_{today}.csv'
filterdf.to_csv(filename, index=False)
print(f"Results saved to {filename}")


Results saved to /Users/nshaffer/Desktop/equity_vol_screen_2025-07-24.csv
